#KAGGLE DOWNLOAD

In [66]:
!pip install --upgrade --force-reinstall --no-deps kaggle==1.5.8
!mkdir /root/.kaggle

with open("/root/.kaggle/kaggle.json", "w+") as f:

    # Enter your kaggle username <USERNAME> and key <KEY> here.
    f.write('{"username":"<USERNAME>","key":"<KEY>"}')


!chmod 600 /root/.kaggle/kaggle.json

  Using cached kaggle-1.5.8-py3-none-any.whl
  Attempting uninstall: kaggle
    Found existing installation: kaggle 1.5.8
    Uninstalling kaggle-1.5.8:
      Successfully uninstalled kaggle-1.5.8
mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [67]:
''' Downloading the dataset. '''

!kaggle competitions download -c eedi-mining-misconceptions-in-mathematics


eedi-mining-misconceptions-in-mathematics.zip: Skipping, found more recently modified local copy (use --force to force download)


In [21]:
''' Unzipping quickly and quietly. '''
!unzip -q eedi-mining-misconceptions-in-mathematics.zip -d ./data

replace ./data/misconception_mapping.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: N


# Overview

- [Infer Notebook](https://www.kaggle.com/code/sinchir0/fine-tuning-bge-infer/notebook)

- make 25 retrieval data by `bge-large-en-v1.5`
- Fine-tuning `bge-large-en-v1.5` by retrieval data
  - `anchor`: `ConstructName` + `SubjectName` + `QuestionText` + `Answer[A-D]Text`
  - `positive`: Correct MisconceptionName
  - `negative`: Wrong MisconceptionName

ref: https://sbert.net/docs/sentence_transformer/training_overview.html#trainer

# Setting

In [68]:
EXP_NAME = "fine-tuning-BGE-Large-pandas"
#DATA_PATH = "/kaggle/input/eedi-mining-misconceptions-in-mathematics"
DATA_PATH = "/content/data"
MODEL_NAME = "BAAI/bge-large-en-v1.5"
#MODEL_NAME = "/kaggle/input/bge-large-polars/pytorch/default/1/trained_model"
COMPETITION_NAME = "eedi-mining-misconceptions-in-mathematics"
OUTPUT_PATH = "."
MODEL_OUTPUT_PATH = f"{OUTPUT_PATH}/trained_model"

RETRIEVE_NUM = 25

EPOCH = 2
LR = 2e-05
BS = 4
GRAD_ACC_STEP = 128 // 4

TRAINING = True
DEBUG = False
WANDB = True

# Install

In [69]:
%pip install -qq datasets
%pip install -qq sentence_transformers
%pip install -qq pandas
%pip install -qq transformers
%pip install -qq wandb

# Import

In [70]:
import os
import numpy as np

from datasets import load_dataset, Dataset

import wandb
import polars as pl
import pandas as pd

import datasets
import sentence_transformers
import transformers

from sklearn.metrics.pairwise import cosine_similarity

from sentence_transformers.losses import MultipleNegativesRankingLoss
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
)
from sentence_transformers.training_args import BatchSamplers
from sentence_transformers.evaluation import TripletEvaluator


In [26]:
NUM_PROC = os.cpu_count()

# WANDB

In [71]:
if WANDB:
    # Settings -> add wandb api
    from google.colab import userdata
    wandb.login(key=userdata.get('wandb_api'))
    wandb.init(project=COMPETITION_NAME, name=EXP_NAME)
    REPORT_TO = "wandb"
else:
    REPORT_TO = "none"

REPORT_TO

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


train/epoch,▁▂▃▄▅▆▇█
train/global_step,▁▂▃▄▅▆▇█
train/grad_norm,██▂▇▃▁▇▇
train/learning_rate,▁██▇▆▅▃▁
train/loss,█▄▄▄▂▁▂▁
train/epoch,0.95841
train/global_step,800
train/grad_norm,8.5015
train/learning_rate,1e-05
train/loss,0.9836


'wandb'

# Data Load

In [72]:
train_pd = pd.read_csv(f"{DATA_PATH}/train.csv")
misconception_mapping_pd = pd.read_csv(f"{DATA_PATH}/misconception_mapping.csv")

In [73]:
train_pd.head()

,QuestionId,ConstructId,ConstructName,SubjectId,SubjectName,CorrectAnswer,QuestionText,AnswerAText,AnswerBText,AnswerCText,AnswerDText,MisconceptionAId,MisconceptionBId,MisconceptionCId,MisconceptionDId
0,0,856,Use the order of operations to carry out calcu...,33,BIDMAS,A,\[\n3 \times 2+4-5\n\]\nWhere do the brackets ...,\( 3 \times(2+4)-5 \),\( 3 \times 2+(4-5) \),\( 3 \times(2+4-5) \),Does not need brackets,NaN,NaN,NaN,1672.0
1,1,1612,Simplify an algebraic fraction by factorising ...,1077,Simplifying Algebraic Fractions,D,"Simplify the following, if possible: \( \frac{...",\( m+1 \),\( m+2 \),\( m-1 \),Does not simplify,2142.0,143.0,2142.0,NaN
2,2,2774,Calculate the range from a list of data,339,Range and Interquartile Range from a List of Data,B,Tom and Katie are discussing the \( 5 \) plant...,Only\nTom,Only\nKatie,Both Tom and Katie,Neither is correct,1287.0,NaN,1287.0,1073.0
3,3,2377,Recall and use the intersecting diagonals prop...,88,Properties of Quadrilaterals,C,The angles highlighted on this rectangle with ...,acute,obtuse,\( 90^{\circ} \),Not enough information,1180.0,1180.0,NaN,1180.0
4,4,3387,Substitute positive integer values into formul...,67,Substitution into Formula,A,The equation \( f=3 r^{2}+3 \) is used to find...,\( 30 \),\( 27 \),\( 51 \),\( 24 \),NaN,NaN,NaN,1818.0


In [74]:
train_pd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1869 entries, 0 to 1868
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   QuestionId        1869 non-null   int64  
 1   ConstructId       1869 non-null   int64  
 2   ConstructName     1869 non-null   object 
 3   SubjectId         1869 non-null   int64  
 4   SubjectName       1869 non-null   object 
 5   CorrectAnswer     1869 non-null   object 
 6   QuestionText      1869 non-null   object 
 7   AnswerAText       1869 non-null   object 
 8   AnswerBText       1869 non-null   object 
 9   AnswerCText       1869 non-null   object 
 10  AnswerDText       1869 non-null   object 
 11  MisconceptionAId  1135 non-null   float64
 12  MisconceptionBId  1118 non-null   float64
 13  MisconceptionCId  1080 non-null   float64
 14  MisconceptionDId  1037 non-null   float64
dtypes: float64(4), int64(3), object(8)
memory usage: 219.1+ KB


In [75]:
train_pd = train_pd.astype({col: 'string' for col in train_pd.select_dtypes(include='object').columns})

In [76]:
common_col = [
    "QuestionId",
    "ConstructName",
    "SubjectName",
    "QuestionText",
    "CorrectAnswer",
]

# Step 1: Select common columns and Answer columns
answer_cols = [f"Answer{alpha}Text" for alpha in ["A", "B", "C", "D"]]
selected_cols = common_col + answer_cols
train_selected = train_pd[selected_cols]

# Step 2: Unpivot (melt) the DataFrame
train_long_pd = train_selected.melt(
    id_vars=common_col,
    value_vars=answer_cols,
    var_name="AnswerType",
    value_name="AnswerText"
)

# Step 3: Create the "AllText" column by concatenating strings
train_long_pd["AllText"] = (
    train_long_pd["ConstructName"] + " " +
    train_long_pd["SubjectName"] + " " +
    train_long_pd["QuestionText"] + " " +
    train_long_pd["AnswerText"]
)

# Step 4: Extract the Answer Alphabet (A, B, C, D) from "AnswerType"
train_long_pd["AnswerAlphabet"] = train_long_pd["AnswerType"].str.extract(r"Answer([A-D])Text$")

# Step 5: Create the "QuestionId_Answer" column
train_long_pd["QuestionId_Answer"] = train_long_pd["QuestionId"].astype("string") + "_" + train_long_pd["AnswerAlphabet"]

# Step 6: Sort by "QuestionId_Answer"
train_long_pd = train_long_pd.sort_values("QuestionId_Answer").reset_index(drop=True)

# Display the first few rows
train_long_pd.head()

,QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer
0,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerAText,\( 3 \times(2+4)-5 \),Use the order of operations to carry out calcu...,A,0_A
1,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerBText,\( 3 \times 2+(4-5) \),Use the order of operations to carry out calcu...,B,0_B
2,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerCText,\( 3 \times(2+4-5) \),Use the order of operations to carry out calcu...,C,0_C
3,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerDText,Does not need brackets,Use the order of operations to carry out calcu...,D,0_D
4,1000,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",B,AnswerAText,\( t \),Simplify an algebraic fraction by factorising ...,A,1000_A


In [77]:
# Define common columns and Misconception columns
misconception_cols = [f"Misconception{alpha}Id" for alpha in ["A", "B", "C", "D"]]

# Step 1: Select the relevant columns
selected_cols = common_col + misconception_cols
train_selected = train_pd[selected_cols]

# Step 2: Unpivot (melt) the DataFrame
train_misconception_long_pd = train_selected.melt(
    id_vars=common_col,
    value_vars=misconception_cols,
    var_name="MisconceptionType",
    value_name="MisconceptionId"
)

# Step 3: Extract the Answer Alphabet (A, B, C, D) from "MisconceptionType"
train_misconception_long_pd["AnswerAlphabet"] = train_misconception_long_pd["MisconceptionType"].str.extract(r"Misconception([A-D])Id$")

# Step 4: Create the "QuestionId_Answer" column
train_misconception_long_pd["QuestionId_Answer"] = (
    train_misconception_long_pd["QuestionId"].astype("string") + "_" + train_misconception_long_pd["AnswerAlphabet"]
)

# Step 5: Select the relevant columns and cast MisconceptionId to int
train_misconception_long_pd = train_misconception_long_pd[["QuestionId_Answer", "MisconceptionId"]]
train_misconception_long_pd["MisconceptionId"] = train_misconception_long_pd["MisconceptionId"].astype('Int64')

# Step 6: Sort by "QuestionId_Answer"
train_misconception_long_pd = train_misconception_long_pd.sort_values("QuestionId_Answer").reset_index(drop=True)

# Display the first few rows
train_misconception_long_pd.head()

,QuestionId_Answer,MisconceptionId
0,0_A,<NA>
1,0_B,<NA>
2,0_C,<NA>
3,0_D,1672
4,1000_A,891


In [78]:
# Perform the join using the "QuestionId_Answer" column
train_long_pd = pd.merge(train_long_pd, train_misconception_long_pd, on="QuestionId_Answer", how="left")

# Display the first few rows
train_long_pd.head()

,QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId
0,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerAText,\( 3 \times(2+4)-5 \),Use the order of operations to carry out calcu...,A,0_A,<NA>
1,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerBText,\( 3 \times 2+(4-5) \),Use the order of operations to carry out calcu...,B,0_B,<NA>
2,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerCText,\( 3 \times(2+4-5) \),Use the order of operations to carry out calcu...,C,0_C,<NA>
3,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerDText,Does not need brackets,Use the order of operations to carry out calcu...,D,0_D,1672
4,1000,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",B,AnswerAText,\( t \),Simplify an algebraic fraction by factorising ...,A,1000_A,891


# Make retrieval data

In [79]:
model = SentenceTransformer(MODEL_NAME, trust_remote_code=True)
train_long_vec_pd = model.encode(
    train_long_pd["AllText"].to_list(), normalize_embeddings=True
)
misconception_mapping_vec_pd = model.encode(
    misconception_mapping_pd["MisconceptionName"].to_list(), normalize_embeddings=True
)
print(train_long_vec_pd.shape)
print(misconception_mapping_vec_pd.shape)

(7476, 1024)
(2587, 1024)


In [87]:
train_cos_sim_arr_pd = cosine_similarity(train_long_vec_pd, misconception_mapping_vec_pd)
train_sorted_indices_pd = np.argsort(-train_cos_sim_arr_pd, axis=1)
print(train_sorted_indices_pd)

[[2306 2586 2518 ... 2461  453 2047]
 [2306 2586 2518 ... 1136  453 2047]
 [2306 2586 2518 ... 2461  453 2047]
 ...
 [1475 2497 1889 ...  906 2299 2461]
 [1475 2497 1889 ...  906 2299 2461]
 [1475 2497 1889 ...  906 2299 2461]]


In [88]:
# Convert the relevant slice to a list and assign it as a new column
train_long_pd["PredictMisconceptionId"] = train_sorted_indices_pd[:, :RETRIEVE_NUM].tolist()

# Display the first few rows
train_long_pd.head()

,QuestionId,ConstructName,SubjectName,QuestionText,CorrectAnswer,AnswerType,AnswerText,AllText,AnswerAlphabet,QuestionId_Answer,MisconceptionId,PredictMisconceptionId
0,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerAText,\( 3 \times(2+4)-5 \),Use the order of operations to carry out calcu...,A,0_A,<NA>,"[2306, 2586, 2518, 987, 2488, 2221, 1119, 1999..."
1,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerBText,\( 3 \times 2+(4-5) \),Use the order of operations to carry out calcu...,B,0_B,<NA>,"[2306, 2586, 2518, 987, 2488, 2221, 1119, 466,..."
2,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerCText,\( 3 \times(2+4-5) \),Use the order of operations to carry out calcu...,C,0_C,<NA>,"[2306, 2586, 2518, 987, 2488, 2221, 1119, 1999..."
3,0,Use the order of operations to carry out calcu...,BIDMAS,\[ 3 \times 2+4-5 \] Where do the brackets nee...,A,AnswerDText,Does not need brackets,Use the order of operations to carry out calcu...,D,0_D,1672,"[2306, 2586, 2488, 2518, 1316, 871, 1119, 466,..."
4,1000,Simplify an algebraic fraction by factorising ...,Simplifying Algebraic Fractions,"Simplify the following, if possible: \( \frac{...",B,AnswerAText,\( t \),Simplify an algebraic fraction by factorising ...,A,1000_A,891,"[979, 2398, 1540, 1593, 363, 2307, 715, 59, 20..."


In [89]:
# Step 1: Filter rows where "MisconceptionId" is not null
train_retrieved_pd = train_long_pd[train_long_pd["MisconceptionId"].notnull()]

# Step 2: Explode the "PredictMisconceptionId" column (assuming it contains lists)
train_retrieved_pd = train_retrieved_pd.explode("PredictMisconceptionId")

# Step 3: Perform a left join with misconception_mapping on "MisconceptionId"
train_retrieved_pd = train_retrieved_pd.merge(misconception_mapping_pd, on="MisconceptionId", how="left")

# Step 4: Perform a left join with renamed misconception_mapping on "PredictMisconceptionId"
misconception_mapping_renamed_pd = misconception_mapping_pd.rename(columns=lambda x: "Predict" + x)
train_retrieved_pd = train_retrieved_pd.merge(misconception_mapping_renamed_pd, on="PredictMisconceptionId", how="left")

# Display the shape and the first few rows
print(train_retrieved_pd.shape)
print(train_retrieved_pd.head())

(109250, 14)
   QuestionId                                      ConstructName SubjectName  \
0           0  Use the order of operations to carry out calcu...      BIDMAS   
1           0  Use the order of operations to carry out calcu...      BIDMAS   
2           0  Use the order of operations to carry out calcu...      BIDMAS   
3           0  Use the order of operations to carry out calcu...      BIDMAS   
4           0  Use the order of operations to carry out calcu...      BIDMAS   

                                        QuestionText CorrectAnswer  \
0  \[
3 \times 2+4-5
\]
Where do the brackets nee...             A   
1  \[
3 \times 2+4-5
\]
Where do the brackets nee...             A   
2  \[
3 \times 2+4-5
\]
Where do the brackets nee...             A   
3  \[
3 \times 2+4-5
\]
Where do the brackets nee...             A   
4  \[
3 \times 2+4-5
\]
Where do the brackets nee...             A   

    AnswerType              AnswerText  \
0  AnswerDText  Does not need brackets   
1

In [90]:
train_retrieved_pd = train_retrieved_pd.reset_index(drop=True)

# Fine-Tune bge

In [91]:
# Assuming `train_retrieved` is a Pandas DataFrame, convert it to a Hugging Face Dataset
train_pd = Dataset.from_pandas(train_retrieved_pd)

# Define the filtering function
def filter_fn(row):
    return row["MisconceptionId"] != row["PredictMisconceptionId"]

# Step 1: Filter the dataset
train_pd = train_pd.filter(filter_fn, num_proc=4)  # Use 4 processors for parallel filtering


/usr/local/lib/python3.10/dist-packages/multiprocess/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()


Filter (num_proc=4):   0%|          | 0/109250 [00:00<?, ? examples/s]

In [92]:
train_pd

Dataset({
    features: ['QuestionId', 'ConstructName', 'SubjectName', 'QuestionText', 'CorrectAnswer', 'AnswerType', 'AnswerText', 'AllText', 'AnswerAlphabet', 'QuestionId_Answer', 'MisconceptionId', 'PredictMisconceptionId', 'MisconceptionName', 'PredictMisconceptionName'],
    num_rows: 106844
})

In [ ]:
if DEBUG:
    train = train_pd.select(range(1000))
    EPOCH = 1

In [93]:
print(f"{sum(p.numel() for p in model.parameters()):,}")

335,141,888


In [ ]:
model = SentenceTransformer(MODEL_NAME)

loss = MultipleNegativesRankingLoss(model)

args = SentenceTransformerTrainingArguments(
    # Required parameter:
    output_dir=OUTPUT_PATH,
    # Optional training parameters:
    num_train_epochs=EPOCH,
    per_device_train_batch_size=BS,
    gradient_accumulation_steps=GRAD_ACC_STEP,
    per_device_eval_batch_size=BS,
    eval_accumulation_steps=GRAD_ACC_STEP,
    learning_rate=LR,
    weight_decay=0.01,
    warmup_ratio=0.1,
    fp16=True,  # Set to False if you get an error that your GPU can't run on FP16
    bf16=False,  # Set to True if you have a GPU that supports BF16
    batch_sampler=BatchSamplers.NO_DUPLICATES,  # MultipleNegativesRankingLoss benefits from no duplicate samples in a batch
    # Optional tracking/debugging parameters:
    lr_scheduler_type="cosine_with_restarts",
    save_strategy="steps",
    save_steps=0.1,
    save_total_limit=2,
    logging_steps=100,
    report_to=REPORT_TO,  # Will be used in W&B if `wandb` is installed
    run_name=EXP_NAME,
    do_eval=False
)

trainer = SentenceTransformerTrainer(
    model=model,
    args=args,
    train_dataset=train_pd.select_columns(
        ["AllText", "MisconceptionName", "PredictMisconceptionName"]
    ),
    loss=loss
)

trainer.train()
model.save_pretrained(MODEL_OUTPUT_PATH)

In [47]:
model.save_pretrained(MODEL_OUTPUT_PATH)

In [51]:
!tar -czf trained_model.tar.gz /content/trained_model/*

tar: Removing leading `/' from member names
tar: Removing leading `/' from hard link targets
